INTRODUCTION


In [ ]:
import os

pwd = os.getcwd()

display(pwd)

import pyspark

# create spqrt context
sc =  pyspark.SparkContext(appName="FichesPratiquesDT")


CREATION DU SCHEMA

CHARGER LE JSON A PARTIR D'UN LIEN

In [ ]:

from pyspark.sql import SparkSession
import requests
import json

# Initialize a Spark session
spark = SparkSession.builder.appName("FichesPratiquesDT").getOrCreate()

# Step 1: Fetch JSON data from the URL
# response = requests.get("http://api.luftdaten.info/static/v1/data.json" )


# URL of the JSON file
url = "https://raw.githubusercontent.com/SocialGouv/fiches-travail-data/master/data/fiches-travail.json"
# Fetch JSON data from the URL
response = requests.get(url)
json_data = response.json()  # Parse the JSON data

# display(json_data[:1])


CHARGER LE JSON A PARTIR DU REPERTOIRE

In [66]:

from pyspark.sql.types import StructType, StructField, StringType, ArrayType, MapType
from pyspark.sql.functions import col

# Define the schema
schema = StructType([
    StructField("date", StringType(), True),
    StructField("description", StringType(), True),
    StructField("intro", StringType(), True),
    StructField("pubId", StringType(), True),
    StructField("section_anchor", StringType(), True),
    StructField("section_html", StringType(), True),
    StructField("section_text", StringType(), True),
    StructField("section_title", StringType(), True),
    StructField("section_description", StringType(), True),
    StructField("section_references", MapType(StringType(), StringType()), True)
])

def flatten_filter_and_map_json_data(records):
    filtered_records = []
    for record in records:
        # Copy all fields except 'sections'
        new_record = {key: value for key, value in record.items() if key != 'sections'}
        
        # Filter 'sections' if it exists and flatten it
        # Ensure sections exist and is a list
        if 'sections' in record and isinstance(record['sections'], list):
            for section in record['sections']:
                if isinstance(section, dict):  # Ensure each section is a dictionary
                    flat_section = {f"section_{key}": value for key, value in section.items() if key in ['title', 'text', 'anchor', 'description']}
                    combined_record = {**new_record, **flat_section}
                    filtered_records.append(combined_record)
        else:
            filtered_records.append(new_record)
    return filtered_records

# Apply the filtering and mapping function
data = flatten_filter_and_map_json_data(json_data)

# Convert the list of dictionaries to JSONLines format
# data = "\n".join([json.dumps(record) for record in data])

# Step 2: Convert JSON data to an RDD
rdd = spark.sparkContext.parallelize(data[:1000])
data = None

# Print the content of the RDD
# for record in rdd.collect():
#     print(record)


# Step 3: Convert RDD to DataFrame
# df = spark.read.json(rdd,  multiLine=True)
# df = spark.read.schema(schema).json(rdd,  multiLine=True)

df = spark.read.json(rdd, multiLine=False)

# Filter out rows with `_corrupt_record` and select only valid rows
df = df.filter(col("_corrupt_record").isNull()).drop("_corrupt_record")

# Print the schema of the DataFrame
# df.printSchema()
# 
# Show the first 2 rows of the DataFrame
df.show(20, truncate=True)

# print df head
# display(df.head(2))



+----------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      date|         description|               intro|        pubId|      section_anchor| section_description|        section_text|       section_title|               title|                 url|
+----------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|02/08/2024|Chaque syndicat r...|<p>Chaque syndica...|article375542|                    |À savoir ! Le man...| À savoir ! Le ma...|Les délégués synd...|Les délégués synd...|https://travail-e...|
|02/08/2024|Chaque syndicat r...|<p>Chaque syndica...|article375542|L-action-des-synd...|L'action des synd...|L'action des synd...|L’action des synd...|Les délégués synd...|https://travail-e...|
|14/06/2024|Le contrat de

In [62]:
from pyspark.sql.functions import col, sum, when, countDistinct

# Print the schema of the DataFrame
print("Schema:")
df.printSchema()

# Show basic statistics for numerical columns
print("\nBasic statistics (describe):")
df.describe().show()

# Show detailed summary statistics
print("\nDetailed summary statistics:")
df.summary().show()

# Count total number of rows
row_count = df.count()
print(f"\nTotal number of rows: {row_count}")


# Check for null values
print("\nNull values count per column:")
df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).show()

# Show column names and data types
print("\nColumn names and data types:")
for col_name, dtype in df.dtypes:
    print(f"{col_name}: {dtype}")

# Count distinct values in each column
print("\nDistinct values count per column:")
df.select([countDistinct(c).alias(c) for c in df.columns]).show()

Schema:
root
 |-- date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- intro: string (nullable = true)
 |-- pubId: string (nullable = true)
 |-- section_anchor: string (nullable = true)
 |-- section_description: string (nullable = true)
 |-- section_text: string (nullable = true)
 |-- section_title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)


Basic statistics (describe):
+-------+----------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|      date|         description|               intro|        pubId|      section_anchor| section_description|        section_text|       section_title|               title|                 url|
+-------+----------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+

# Write the DataFrame

In [68]:
# Write the DataFrame to a CSV file
df.write.mode("overwrite").option("header", "true").csv("output.csv")

# Save the DataFrame to a JSON file without truncation
df.write.mode("overwrite").json("output.json")

# If you need a single JSON file:
df.coalesce(1).write.mode("overwrite").json("single_output.json")

# Save the DataFrame as a JSONL file
df.write.mode("overwrite").json("output.jsonl", lineSep="\n")

In [ ]:


# Define the schema
schema = StructType([
    StructField("date", StringType(), True),
    StructField("description", StringType(), True),
    StructField("intro", StringType(), True),
    StructField("pubId", StringType(), True),
    StructField("sections", ArrayType(
        StructType([
            StructField("anchor", StringType(), True),
            StructField("html", StringType(), True),
            StructField("text", StringType(), True),
            StructField("title", StringType(), True),
            StructField("description", StringType(), True),
            StructField("references", MapType(StringType(), StringType()), True)
        ])
    ), True)
])

# mapping and filtering
def filter_and_map_json_data(records):
    filtered_records = []
    for record in records:
        # Copy all fields except 'sections'
        new_record = {key: value for key, value in record.items() if key != 'sections'}
        
        # # Filter 'sections' if it exists
        # if 'sections' in record:
        #     new_record['sections'] = [
        #         {key: value for key, value in section.items() if key in ['title', 'text', 'anchor', 'description']}
        #         for section in record['sections']
        #     ]
        
        filtered_records.append(new_record)
    return filtered_records

# Apply the filtering and mapping function
filtered_data = filter_and_map_json_data(json_data)

# Step 2: Convert JSON data to an RDD
rdd = spark.sparkContext.parallelize([filtered_data[:1]])

# Step 3: Convert RDD to DataFrame
df = spark.read.json(rdd,  multiLine=True)
# df = spark.read.schema(schema).json(rdd,  multiLine=True)

# Print the schema of the DataFrame
df.printSchema()

# print df head
display(df.head(2))
